In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

list_dir = []
for dirname, _, filenames in os.walk('/kaggle/input/ulasan'):
    for filename in filenames:
        ld = os.path.join(dirname, filename)
        list_dir.append(ld)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
def get_files(list_dir):

    df_list = pd.DataFrame(columns=["review","extractive","abstractive"])
    review = []
    extractive = []
    abstractive = []
    for f in list_dir:
        print('file ' + str(f))
        data = pd.read_csv(f, encoding="ISO-8859-1", sep=',', usecols=['review','extractive', 'abstractive'])
        data_length = len(data)
        data = data.astype(str)

        str_rev = ""
        str_ext = ""
        for d in range(data_length):
            str_rev += " " + data['review'][d]
            str_ext += " " + data['extractive'][d]
      
        review.append(str_rev)
        extractive.append(str_ext)
        abstractive.append(data['abstractive'][0])
        
    df_list['review'] = review
    df_list['extractive'] = extractive
    df_list['abstractive'] = abstractive

    print(len(df_list))
    return df_list

In [ ]:
data = get_files(list_dir)

In [ ]:
data

In [ ]:
!pip install Sastrawi
import nltk
from nltk.corpus import stopwords
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import itertools
nltk.download('stopwords')
!pip install langdetect
!pip install google_trans_new

In [ ]:
# stop_id = StopWordRemoverFactory().get_stop_words()
stop_factory = stopwords.words('indonesian')
# more_stopword = ['daring', 'online', 'pd']


# https://en.wikipedia.org/wiki/Unicode_block
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "]+"
)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stopword
def get_file(file):
    stopwords=[]
    file_stopwords = open(file,'r')
    row = file_stopwords.readline()
    while row:
        word = row.strip()
        stopwords.append(word)
        row = file_stopwords.readline()
    file_stopwords.close()
    return stopwords

def first_process(text):
    text = text.lower()                               # change to lower case
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # delete URL
    text = re.sub(r'[-+]?[0-9]+', '', text)           # delete number
    text = re.sub(r'[^\w\s\.]',' ', text)        # delete punctuation
    text = re.sub(r'  ',' ', text)                    # delete more space
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.strip()
    return text

#normalize text
def text_normalize(key_file, text):
    key_norm = pd.read_csv(key_file, index_col='_id')
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
    text = str.lower(text)
    return text

#stopword removal
def remove_stop_words(text):
    # sw = stop_factory + more_stopword
    sw = stop_factory
    clean_words = []
    text = text.split()
    for word in text:
        if word not in sw:
            clean_words.append(word)
    return " ".join(clean_words)

def remove_nan(text):
  clean_words = []
  text = text.split()
  for word in text:
    if str(word) != 'nan':
      clean_words.append(word)
  return " ".join(clean_words)

#remove emoji
def emoji(text):
    return re.sub(EMOJI_PATTERN, r"", text)

#delete duplicate char
def duplicate_char(text, key_word):
    text = text.split()
    clean_words = []
    for word in text:
        text_s = stemmer.stem(word)
        tmp = []
        if text_s not in key_word:
            tmp = ''.join(ch for ch, _ in itertools.groupby(word))
            clean_words.append(tmp)
        else:
            clean_words.append(word)
    return " ".join(clean_words)
    
# all process
def clean_process(text):
    text = first_process(text)
    # text = text_normalize('/content/gdrive/MyDrive/Colab Notebooks/Thesis/data/key_norm_new.csv',text)
    # text = remove_stop_words(text)
    text = remove_nan(text)
    text = emoji(text)
    basic_text = get_file('/kaggle/input/corpus/prepro/kata-dasar.txt')
    text = duplicate_char(text,basic_text)
    return text
    
data['review_cln'] = data['review'].apply(clean_process)

In [ ]:
data['review_cln'][0]

In [ ]:
data['extractive_cln'] = data['extractive'].apply(clean_process)

In [ ]:
import re
from tqdm import tqdm


def split_sent(data):
    sentence_delimiters = re.compile(u'[.\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')

    df_as = []

    for i in tqdm(data):
      x = ""
      x = sentence_delimiters.split(i)
      x = [sen.rstrip().lstrip() for sen in x if sen and not sen.isspace()]
      df_as.append(x)
    
    return df_as

In [ ]:
df_as = split_sent(data['review_cln'])
data['sent_tok'] = df_as

In [ ]:
len(data['sent_tok'])

In [ ]:
# df_ex = split_sent(data['extractive_cln'])
# data['ex_tok'] = df_ex

In [ ]:
data

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertModel.from_pretrained('indobenchmark/indobert-base-p1', output_hidden_states = True)

In [ ]:
model

In [ ]:
use_gpu = False # Use GPU for text vectorization
n_combined_hidden_states = 1 # Number of last hidden states for combining to single vector
pooling_mode = 0 # pooling strategy for BERT-embeddings

if use_gpu:
    model.to('cuda')

def sentence2bert_hidden_states(sentence, tokenizer, model, use_gpu = True):
    indexed_tokens = tokenizer.encode(sentence)
    tokens_tensor = torch.tensor([indexed_tokens])
    if use_gpu:
        tokens_tensor = tokens_tensor.to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)        
    hidden_states = outputs[2]
    hidden_states = torch.stack(hidden_states, dim=0)
    hidden_states = torch.squeeze(hidden_states, dim=1)
    hidden_states = hidden_states.permute(1,0,2)    
    return hidden_states, indexed_tokens

def combine_hidden_states(hidden_states, n_summands):
    n = hidden_states.shape[0]    
    assert n_summands <= hidden_states.shape[1]    
    combined = np.empty((hidden_states.shape[0],hidden_states.shape[2]))
    for i in range(n):
        combined[i,:] = torch.sum(hidden_states[i][-n_summands:], dim=0).cpu()[:]
    return combined

def sentence2token_embeddings(sentence, tokenizer, model, use_gpu = True, n_combined_hidden_states = 4):
    hidden_states, indexed_tokens = sentence2bert_hidden_states(sentence, tokenizer, model, use_gpu)
    token_embeddings = combine_hidden_states(hidden_states, n_combined_hidden_states)
    return token_embeddings, indexed_tokens

# Sentence vectorization
def sentence2embedding(sentence, tokenizer, model, use_gpu = True, n_combined_hidden_states = 4, pooling_mode = 0):
    token_embeddings, indexed_tokens = sentence2token_embeddings(sentence, tokenizer, model, use_gpu, n_combined_hidden_states)
    if pooling_mode == 0:
        return token_embeddings[0]
    else:
        return np.mean(token_embeddings, axis = 0)

# Splitting text into sentences and sentence by sentence vectorization
def text2sentence_embeddings(text, tokenizer, model, use_gpu = True, n_combined_hidden_states = 4, pooling_mode = 0):
    text_sentences = text#nltk.sent_
    n_text_sentences = len(text_sentences)
    text_embeddings = np.empty((n_text_sentences, model.config.hidden_size))
    for i in tqdm(range(n_text_sentences)):
        text_embeddings[i,:] = sentence2embedding(text_sentences[i], tokenizer, model, use_gpu, n_combined_hidden_states, pooling_mode)[:]
    return text_sentences, text_embeddings

In [ ]:
import numpy as np

def sentence_embeddings(data):
  text_sentences, text_embeddings = text2sentence_embeddings(data, 
                                                            tokenizer, 
                                                            model, 
                                                            use_gpu, 
                                                            n_combined_hidden_states, 
                                                            pooling_mode)
  return text_sentences, text_embeddings

In [ ]:
def cosine(u, v):
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
!pip install scikit-learn-extra

In [ ]:
!pip install yellowbrick

In [ ]:
import numpy as np

def gap_statistic(embedding, k_max, nrefs=10):
  gaps = np.zeros((len(range(2, k_max)),))
  data_max = np.amax(embedding[:,1])
  data_min = np.amin(embedding[:,1])

  for gap_index, k in enumerate(range(2, k_max)):
    refDisps = np.zeros(nrefs)

    for i in range(nrefs):
      reference = np.random.uniform(data_min, data_max, embedding.shape[0]).reshape(-1,1)
      kmed_model = KMedoids(k)
      kmed_model.fit(reference)
      refDisp = kmed_model.inertia_
      refDisps[i] = refDisp
    
    kmed_model = KMedoids(k)
    kmed_model.fit(embedding)
        
    origDisp = kmed_model.inertia_

    gap = np.log(np.mean(refDisps)) - np.log(origDisp)
    gaps[gap_index] = gap

  return (gaps.argmax() + 2)

# text, embedding = sentence_embeddings(df['sent_tok'][0])
# pca = PCA(n_components=2)
# embedding = pca.fit_transform(embedding)
# s = gap_statistic(embedding, 20)
# print(s)

In [ ]:
from sklearn.metrics import silhouette_score

def get_silhouette (embedding, k_max):
  silhouette = {}
  for n_cluster in range(2, k_max):
    silhouette[n_cluster] = silhouette_score(embedding, KMedoids(n_clusters = n_cluster).fit_predict(embedding))
  
  index_max = max(silhouette, key=silhouette.get)
  return index_max

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

def elbow(embedding, k_max):
  kmed_model = KMedoids()
  elb_visualizer = KElbowVisualizer(kmed_model, k=(2,k_max), timings=False)
  elb_visualizer.fit(embedding)
  index_clus = elb_visualizer.elbow_value_

  return index_clus

In [ ]:
from sklearn.decomposition import PCA
from sklearn_extra.cluster import KMedoids


def n_clustering(embedding, k_max):
    pca = PCA(n_components=2)
    embedding = pca.fit_transform(embedding)
    clus = []

  # for n_cluster in range(2, len(embedding)):
  #   silhouette_scores[n_cluster] = silhouette_score(embedding, KMedoids(n_clusters = n_cluster).fit_predict(embedding))
  #   # silhouette_scores.append(silhouette_score(embedding,
  #   #                                           AgglomerativeClustering(n_clusters = n_cluster).fit_predict(embedding))) 
  
  # index_max = max(silhouette_scores, key=silhouette_scores.get)
#   k_max = 10
#   print(k_max)
    gs = gap_statistic(embedding, k_max)
    ss = get_silhouette(embedding, k_max)
    el = elbow(embedding, k_max)
    
    clus = [gs, ss, el]
    
    for cl in clus:
        if cl is None:
            clus.remove(cl)
            
    print(clus)
    clus.sort()
    clus_opt = {x:clus.count(x) for x in clus}
    index_max = max(clus_opt, key=clus_opt.get)
    print(index_max)

    return index_max

def clustering(embedding,n_cluster):
    if n_cluster >= 1:
        n_cluster = np.round(n_cluster)
    else:
        n_cluster = 1

    agg = KMedoids(n_clusters=n_cluster)
    agg.fit(embedding)

    return agg

In [ ]:
from collections import defaultdict

def clustering_sent(data, cls_max=False):
    data_ext = defaultdict(dict)
    count = 0

#     for i in data:
#       text, embedding = sentence_embeddings(i)
#       tmp_txt = []
#       tmp_clus = []
#       tmp_centroid = []
#       if len(embedding) <= 6:
#         for i in range(len(text)):
#           tmp_txt.append(text[i])
#           tmp_clus.append(0)
#       else:
#         cls = n_clustering(embedding)
#         agg = clustering(embedding,cls)
#         for i in range(cls):
#           tmp_centroid.append(agg.cluster_centers_[i])
#         for i in range(len(text)):
#           tmp_txt.append(text[i])
#           tmp_clus.append(agg.labels_[i])
#       data_ext[count]['text'] = tmp_txt
#       data_ext[count]['cluster'] = tmp_clus
#       data_ext[count]['centroid'] = tmp_centroid
#       count += 1
        
    for i in data:
        text, embedding = sentence_embeddings(i)
        tmp_txt = []
        tmp_clus = []
        tmp_centroid = []
        
        if cls_max == False:
            k_max = 10
        else:
            k_max = len(i)
        
        cls = n_clustering(embedding, k_max)
        agg = clustering(embedding,cls)

        for i in range(len(text)):
            tmp_txt.append(text[i])
            tmp_clus.append(agg.labels_[i])
            
        data_ext[count]['text'] = tmp_txt
        data_ext[count]['cluster'] = tmp_clus
        count += 1
    
    return data_ext

In [ ]:
data_ext = clustering_sent(data['sent_tok'], cls_max=True)

In [ ]:
len(data_ext)

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize


def convert_to_embeddings(text_sentences, text_embeddings, text, lexicon, keyword):
  # text = " ".join(data)
  # print(text)
  hypothesis_embeddings = {}
  sum = 0
  index = 0

  # num_clusters = 6
  # clustering_model = KMeans(n_clusters=num_clusters)
  # clustering_model.fit(text_embeddings)
  # cluster_assignment = clustering_model.labels_

  # clustered_sentences = [[] for i in range(num_clusters)]
  # for sentence_id, cluster_id in enumerate(cluster_assignment):
  #     clustered_sentences[cluster_id].append(df['sent_tok'][0][sentence_id])
  for sent in text_sentences:
    var = text_embeddings[index]
    sum += var
    hypothesis_embeddings[sent] = var
    index += 1

  centroid = sum/len(text_sentences)
  
  #print("done")
  #print(hypothesis_embeddings)
  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  # sentence_position_score = sentence_position(hypothesis)
  sentiment = softmax_sen(text, lexicon, keyword)
  return total_score(content_relevance_score, sentence_novelty_score, sentiment)

In [ ]:
def content_relevance(centroid,hypothesis_embeddings):
  d = {}
  for sentence,embed in hypothesis_embeddings.items():
    d[sentence] = cosine(centroid,embed)
    
  return d

In [ ]:
def sentence_novelty(content_relevance_score,hypothesis_embeddings):
  novel_sentences = {}
  TAU = 0.70
  for sent1,embed1 in hypothesis_embeddings.items():
    max_similarity = 0
    for sent2,embed2 in hypothesis_embeddings.items():
      if sent1!=sent2 and cosine(embed1,embed2)>max_similarity:
          max_similarity = cosine(embed1,embed2)

    if max_similarity<TAU:
      novel_sentences[sent1] = 1

    elif max_similarity>TAU and content_relevance_score[sent1]>content_relevance_score[sent2]:
      novel_sentences[sent1] = 1

    else:
      novel_sentences[sent1] = 1-max_similarity

  return novel_sentences

In [ ]:
!pip install git+https://github.com/LIAAD/yake

In [ ]:
def key_clean_process(text):
    text = remove_stop_words(text)
    return text

data['review_key'] = data['review'].apply(key_clean_process)

In [ ]:
data['review_key'][0]

In [ ]:
import yake

def keyword_extractor(text):
    kword = []
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(text)
    
    for kw in range(len(keywords)):
        if len(keywords[kw][0].split()) >= 2:
            kword.append(keywords[kw])
    
    keywords = sorted(kword,key=lambda x:(-x[1],x[0]))

    return keywords

In [ ]:
keyword = []
for i in data['review_key']:
  keyword.append(keyword_extractor(i))

In [ ]:
data

In [ ]:
for i in range(len(data)):
  data_ext[i]['keyword'] = keyword[i]

In [ ]:
data_ext[0]['keyword']

In [ ]:
from numpy import exp

# Fungsi Load Lexicon
def loadLexicon(file):
    df=open(file,"r",encoding="utf-8", errors='replace')
    data=df.readlines();df.close()
    return [d.strip().lower() for d in data]

# fpos, fneg, fnegasi = '/content/gdrive/MyDrive/dataset/sentimen/positive.tsv', '/content/gdrive/MyDrive/dataset/sentimen/negative.tsv', '/content/gdrive/MyDrive/dataset/sentimen/s-negasi.txt'
# # positif, negatif, negasi = loadLexicon(fpos), loadLexicon(fneg), loadLexicon(fnegasi)
# pos_lexicon = pd.read_csv('lexicon/InSet-master/positive.tsv',sep='\t')
# neg_lexicon = pd.read_csv('lexicon/InSet-master/negative.tsv',sep='\t')
# lexicon = pos_lexicon.append(neg_lexicon,ignore_index=True)

 
# calculate the softmax of a vector
def softmax(vector):
	e = exp(vector)
	return e / e.sum()

def prediksiSentiment(kalimat, lexicon, keyword, index):
    # Naive Approach, nanti akan kita diskusikan bagaimana improvisasi fungsi sederhana ini
    # posWords = []
    # negWords = [w for w in negatif if w in kalimat]
    # for w in positif:
    #     if w in kalimat:
    #         negated = True
    #         for n in negasi:
    #             if n+' '+w in kalimat:
    #                 negWords.append(n+' '+w)
    #                 negated = True
    #                 break
    #         if not negated:
    #             posWords.append(w)
    # nPos, nNeg = len(posWords), len(negWords)
    # print('posWords' + str(nPos))
    # print('negWords' + str(nNeg))

    add_weight = []
    for i in range(0,len(lexicon)):
      if lexicon['word'][i] in kalimat:
        add_weight.append(lexicon['weight'][i])
    val = sum(add_weight)
    wordlist = len(kalimat.split())

    nk = 0
    for k in range(len(keyword)):
      if keyword[k][0] in kalimat:
        nk += keyword[k][1]
        

    return val, wordlist, nk

In [ ]:
def softmax_sen(text, lexicon, keyword):
  lprob = []
  clus_sent = {}
  index = 0
  for i,sent in enumerate(text):
    val, wordlist, key = prediksiSentiment(sent, lexicon, keyword, i)
    try:
      ns = (val / wordlist)
    except:
      ns = 0
    prob = ns + key
    lprob.append(prob)
    clus_sent[sent] = val
    index += 1
  
  sofProb = softmax(lprob)
  score_sent = {}
  for i,sent in enumerate(text):
    score_sent[sent] = sofProb[i]
    # print(sofProb[i],sent)
  
  return score_sent

In [ ]:
def total_score(content_relevance_score, sentence_novelty_score, sentiment):
  ALPHA = 0.45
  BETA = 0.35
  GAMMA = 0.20
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = ALPHA*content_relevance_score[sent]+BETA*sentence_novelty_score[sent]+GAMMA*sentiment[sent]
    print(final_score[sent])
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}

#   summary = ""
  
#   index = 0
#   if len(final_score) <= 5:
#     for sent in final_score:
#       if index == (len(final_score)-1):
#         summary += sent + "."
#       else :
#         summary += sent + ". "
#       index += 1
#   else:
#     for sent in final_score:
#       if final_score[sent] >= threshold: #making summary of approx half length.
#         if index == (len(final_score)-1):
#           summary += sent + "."
#         else :
#           summary += sent + ". "
#       index += 1


  # if len(final_score) > 1:
  #   index = 0

  #     if index == 0:
  #       summary += sent
  #     else:
  #       summary += '. ' + sent
  #   # for i in enumerate(final_score):
  #   #   print(i)
  #     index += 1
  # else:
  #   for sent in final_score:
  #     summary = sent
  # contra = False
  # if len(final_score)>1:
  #   clus = []
  #   for i,sent in enumerate(final_score):
  #     clus.append(cluster[sent])
  #     # print(clus[i])

  #   for i in range(1):
  #     # if clus[i] == 0:
  #     #   break
  #     if clus[i] != clus[i+1]:
  #       # if clus[i+1] == 0:
  #       #   break
  #       contra = True
      
  return final_score

In [ ]:
def iter_ext(data, lexicon):
    extractive = []
    index = 0
    
    result_dict = defaultdict(list)

    while index < len(data):
        total = 0
        print('data ke ' + str(index))
        e_clus = defaultdict(list)
        total = np.array(data[index]['cluster'])

        datapoint_length = {x:data[index]['cluster'].count(x) for x in data[index]['cluster']}
        dat_list = list(datapoint_length.values())

        for c in range(len(datapoint_length)):
            print('cluster ' + str(c))
            txt = []

            for t in range (len(data[index]['text'])):
                if data[index]['cluster'][t] == c:
                    txt.append(data[index]['text'][t])
        
      
            print(len(txt))
            text_sentences, text_embeddings = sentence_embeddings(txt)
            predicted_summary = convert_to_embeddings(text_sentences, text_embeddings, data[index]['text'], lexicon, data[index]['keyword']) 
            e_clus[c].append(predicted_summary)
        
        result_dict[index].append(e_clus)
#         output = ""
#         for v in e_clus:
#             output += v 
#         extractive.append(output)
        index += 1

    return result_dict

In [ ]:
fpos, fneg, fnegasi = '/kaggle/input/corpus/sentimen/positive.tsv', '/kaggle/input/corpus/sentimen/negative.tsv', '/kaggle/input/corpus/sentimen/s-negasi.txt'
# positif, negatif, negasi = loadLexicon(fpos), loadLexicon(fneg), loadLexicon(fnegasi)
pos_lexicon = pd.read_csv(fpos,sep='\t')
neg_lexicon = pd.read_csv(fneg,sep='\t')
lexicon = pos_lexicon.append(neg_lexicon,ignore_index=True)

extract = iter_ext(data_ext, lexicon)

In [ ]:
def summary(text, threshold):
    extractive = []
    for dc in range(len(text)):
        tmp_ext = ''
        for cl in range(len(text[dc][0])):
            val_cls = [v for v in text[dc][0][cl][0].values()]
            key_cls = [k for k in text[dc][0][cl][0].keys()]
            for vc in range(len(val_cls)):
                if val_cls[vc] >= threshold:
                    if len(tmp_ext) == 0:
                        tmp_ext += key_cls[vc] + '.'
                    else:
                        tmp_ext += ' ' + key_cls[vc] + '.'
        extractive.append(tmp_ext)
    
    return extractive

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge
#rouge metric based on https://pypi.org/project/rouge/
def my_rouge2(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    
    #rouge-1
    r1_f1=scores[0]['rouge-1']['f']
    r1_precision=scores[0]['rouge-1']['p']
    r1_recall=scores[0]['rouge-1']['r']
    
    #rouge-2
    r2_f1=scores[0]['rouge-2']['f']
    r2_precision=scores[0]['rouge-2']['p']
    r2_recall=scores[0]['rouge-2']['r']
    
    return r1_f1#, r2_f1, #, precision, recall

In [ ]:
step = 0.5
result_treshold = {}
while step <= 0.95:
    print("="*5 + str(step) + "="*5)
    val_th = []
    
    extractive = summary(extract, step)

    for i in range(len(extractive)):
        if len(extractive[i].split()) != 0:
            tmp_val = my_rouge2(extractive[i], data['abstractive'][i])
            val_th.append(tmp_val)
        else:
            tmp_val = 0
            val_th.append(tmp_val)
  
    tot = sum(val_th)/len(val_th)
    result_treshold[step] = tot

    step += 0.01
    step = round(step,2)

In [ ]:
result_treshold

In [ ]:
final_summary = summary(extract, 0.77)

In [ ]:
data['proposed_extractive'] = final_summary

In [ ]:
data.to_csv(f"/kaggle/working/extractive.csv", index=False, encoding='utf-8')

BERT2GPT2

In [3]:
import string
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import unicodedata

# Remove puncuation from word
def rm_punc_from_word(word):
    clean_alphabet_list = [
        alphabet for alphabet in word if alphabet not in string.punctuation
    ]
    return ''.join(clean_alphabet_list)

# Remove puncuation from text
def rm_punc_from_text(text):
    clean_word_list = [rm_punc_from_word(word) for word in text]
    return ''.join(clean_word_list)

# Remove numbers from text
def rm_number_from_text(text):
    text = re.sub('[0-9]+', '', text)
    return ' '.join(text.split())  # to rm `extra` white space

# Remove stopwords from text
def rm_stopwords_from_text(text):
    _stopwords = stopwords.words('indonesian')
    text = text.split()
    word_list = [word for word in text if word not in _stopwords]
    return ' '.join(word_list)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Cleaning text
def clean_text(text):
    text = text.lower()
    # text = rm_punc_from_text(text)
    text = rm_number_from_text(text)
    # text = rm_stopwords_from_text(text)

    # there are hyphen(–) in many titles, so replacing it with empty str
    # this hyphen(–) is different from normal hyphen(-)
    text = re.sub('–', '', text)
    text = ' '.join(text.split())  # removing `extra` white spaces

    # Removing unnecessary characters from text
    text = re.sub("(\\t)", ' ', str(text)).lower()
    text = re.sub("(\\r)", ' ', str(text)).lower()
    text = re.sub("(\\n)", ' ', str(text)).lower()

    # remove accented chars ('Sómě Áccěntěd těxt' => 'Some Accented text')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode(
        'utf-8', 'ignore'
    )

    text = re.sub("(__+)", ' ', str(text)).lower()
    text = re.sub("(--+)", ' ', str(text)).lower()
    text = re.sub("(~~+)", ' ', str(text)).lower()
    text = re.sub("(\+\++)", ' ', str(text)).lower()
    text = re.sub("(\.\.+)", ' ', str(text)).lower()

    text = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(text)).lower()

    text = re.sub("(mailto:)", ' ', str(text)).lower()
    text = re.sub(r"(\\x9\d)", ' ', str(text)).lower()
    text = re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(text)).lower()
    text = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM',
                  str(text)).lower()

    text = re.sub("(\.\s+)", ' ', str(text)).lower()
    text = re.sub("(\-\s+)", ' ', str(text)).lower()
    text = re.sub("(\:\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(text))
        repl_url = url.group(3)
        text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', repl_url, str(text))
    except Exception as e:
        pass

    text = re.sub("(\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    return text

In [5]:
import pandas as pd

data = pd.read_csv('/kaggle/input/extractive/extractive.csv')
data['abstractive'] = data['abstractive'].apply(clean_text)

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

In [7]:
!pip install datasets
from datasets import Dataset
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

In [9]:
!pip install transformers
from transformers import BertTokenizer, GPT2Tokenizer

VersionConflict: tokenizers==0.9.4 is required for a normal functioning of this module, but found tokenizers==0.12.1.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git master

In [ ]:
tokenizer_src = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
tokenizer_src.bos_token = tokenizer_src.cls_token
tokenizer_src.eos_token = tokenizer_src.sep_token

In [ ]:
# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

In [ ]:
GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
tokenizer_tgt = GPT2Tokenizer.from_pretrained("cahya/gpt2-small-indonesian-522M")
tokenizer_tgt.pad_token = tokenizer_tgt.unk_token

In [ ]:
import matplotlib.pyplot as plt

text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['proposed_extractive']:
      text_word_count.append(len(i.split()))

for i in data['abstractive']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'ulasan':text_word_count, 'ringkasan':summary_word_count})

length_df.hist(bins = 30)
plt.show()

In [ ]:
max_ulasan = max(text_word_count)
print(max_ulasan)
max_ringkasan = max(summary_word_count)
print(max_ringkasan)

In [ ]:
encoder_max_length = max_ulasan
decoder_max_length = max_ringkasan

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer_src(batch["proposed_extractive"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer_tgt(batch["abstractive"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["labels"] = outputs.input_ids.copy()
  batch["decoder_attention_mask"] = outputs.attention_mask

  batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
  
  assert all([len(x) == encoder_max_length for x in inputs.input_ids])
  assert all([len(x) == decoder_max_length for x in outputs.input_ids])
  return batch

In [ ]:
batch_size=32

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["review", "extractive", "review_cln", "extractive_cln", "sent_tok", "review_key"]
)
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["review", "extractive", "review_cln", "extractive_cln", "sent_tok", "review_key"]
)

In [ ]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
train_data[0]

In [ ]:
from transformers import EncoderDecoderModel

bert2gpt = EncoderDecoderModel.from_encoder_decoder_pretrained("indobenchmark/indobert-base-p1","cahya/gpt2-small-indonesian-522M")

In [ ]:
bert2gpt.config.decoder_start_token_id = tokenizer_tgt.bos_token_id
bert2gpt.config.eos_token_id = tokenizer_tgt.eos_token_id
bert2gpt.config.pad_token_id = tokenizer_tgt.pad_token_id
bert2gpt.config.max_length = 20
bert2gpt.config.min_length = 3
bert2gpt.config.no_repeat_ngram_size = 3
bert2gpt.early_stopping = True
bert2gpt.length_penalty = 2.0
bert2gpt.num_beams = 8

In [ ]:
!pip install rouge_score
from datasets import load_metric

rouge = load_metric("rouge")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    #preds, labels = eval_preds
    # if isinstance(pred_ids, tuple):
    #     pred_ids = pred_ids[0]
    # for pred in pred_ids:
    #     print("PRED BEFORE DECODING:", pred)

    # all unnecessary tokens are removed
    pred_str = tokenizer_tgt.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer_tgt.eos_token_id
    label_str = tokenizer_tgt.batch_decode(labels_ids, skip_special_tokens=True)
    # decoded_preds, decoded_labels = postprocess_text(pred_str, label_str)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # fp16=True, 
    output_dir="./",
    do_train=True,
    do_eval=True,
    logging_steps=50,  
    save_steps=100, 
    # warmup_steps=50,  
    # max_steps=1500, # delete for full training
    num_train_epochs = 64, #TRAIN_EPOCHS
    # save_total_limit=1,
)

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2gpt,
    tokenizer=tokenizer_tgt,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

In [ ]:
trainer.save_model('/kaggle/working/bert2gpt')

In [ ]:
model = EncoderDecoderModel.from_pretrained('/kaggle/working/bert2gpt')

In [ ]:
test_data = Dataset.from_pandas(test_df)

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer_src(batch["proposed_extractive"], padding="max_length", truncation=True, max_length=70, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=10,
                                repetition_penalty=5.0, num_return_sequences=1)

    # all special tokens including will be removed
    output_str = tokenizer_tgt.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["review", "extractive", "review_cln", "extractive_cln", "sent_tok", "review_key"])

pred_str = results["pred"]
label_str = results["abstractive"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

In [ ]:
for i in range(len(test_data)):
    print('ulasan: ',test_data[i]['proposed_extractive'])
    print('ringkasan: ',label_str[i])
    print('prediksi: ',pred_str[i])
    print("===================")